<a href="https://colab.research.google.com/github/datascience-uniandes/data-quality-tutorial/blob/master/data-quality-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality and Cleanliness

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
**Dataset:** Homicides Colombia ([datos.gov.co](datos.gov.co))
  
Last update: September, 2023

In [1]:
!pip install pylev

In [2]:
import re
from random import randint
from datetime import datetime
from difflib import SequenceMatcher

import numpy as np
import pandas as pd

import pylev

In [43]:
pd.set_option("display.max_columns", None)

## 1. Loading the data

In [44]:
homicides_df = pd.read_csv("./data/homicides.csv")

In [45]:
homicides_df.shape

(12400, 22)

In [46]:
homicides_df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
HORA                  object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [47]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
6736,07/18/2021 12:00:00 AM,CALDAS,LA DORADA,Sábado,20:10,LAS FERIAS,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,20.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,17380000.0,1995.0,77-656,glmoid7510@gmail.com
8544,09/12/2021 12:00:00 AM,ARAUCA,ARAUQUITA,Sábado,19:30,VEREDA EL TRONCAL,RURAL,FINCAS Y SIMILARES,ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,NO REPORTADO,NO REPORTADO,81065000.0,1970.0,38-897,cphqac7647@gmail.com
1114,02/01/2021 12:00:00 AM,ATLÁNTICO,BARRANQUILLA (CT),Domingo,12:30,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,140-17,correo5853@colombia.gov.co
492,01/13/2021 12:00:00 AM,ANTIOQUIA,FRONTINO,Martes,8:35,ALTO DE LA VIRGEN,RURAL,FINCAS Y SIMILARES,ARMA DE FUEGO,A PIE,A PIE,25.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,5284000.0,1990.0,78-329,mehhkr1492@gmail.com
11896,12/20/2021 12:00:00 AM,GUAJIRA,MAICAO,Doningo,17:30,CENTRO,URBANA,LOCAL COMERCIAL,ARMA DE FUEGO,A PIE,A PIE,47.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,44430000.0,1968.0,95-355,qcutml7077@gmail.com


## 2. Working with datetimes

In [48]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [49]:
# Applying the validation to all values in the column
homicides_df["FECHA"].apply(dateparse)

# IT IS EXPECTED TO HAVE AN ERROR BECAUSE SOME VALUES DOESN'T FIT THE FORMAT

ValueError: ignored

In [50]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [51]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
486,13/12/2021 12:00:00 AM,VALLE,CALI (CT),kunes,23:00,MOJICA E15,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,26.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,89.0,80-330,lujhdf9132@gmail.com
695,30/01/2021 12:00:00 AM,BOLÍVAR,CARTAGENA (CT),Lunes,5:30,REP. DEL LIBANO,URBANA,VIAS PUBLICAS,ARMA BLANCA,NO REPORTADO,A PIE,25.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,13001000.0,1990.0,12-915,ghumtg4094@unidatos.edu.co
1250,18/05/2021 12:00:00 AM,HUILA,TESALIA,Jueves,19:30,VEREDA PACARNI,RURAL,CASAS DE HABITACION,ARMA DE FUEGO,A PIE,A PIE,34.0,FEMENINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,41797000.0,1981.0,99-095,sdaggf6639@gmail.com
12168,12/25/2021 12:00:00 MM,VALLE,PALMIRA,Viernes,15:30,LA EMILIA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,CONDUCTOR MOTOCICLETA,17.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1998.0,16-362,uschca1775@gmail.com
12399,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# Deleting a row by its index
homicides_df.drop([486, 695, 1250, 12168, 12399], inplace=True)

In [53]:
# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [54]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
HORA                         object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [55]:
# Counting homicides by hour
homicides_df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

12    12395
Name: FECHA, dtype: int64

*The homicide hour is available in a different column!*

In [56]:
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11])+homicides_df["HORA"]

In [57]:
# Deleting redundant column
homicides_df.drop(columns=["HORA"], inplace=True)

In [58]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [59]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [60]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [61]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
6447,2021-07-09 16:30:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueves,CARACOLÍ ENO REPORTADO19,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,30.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1985.0,28-192,mramrj9099@gmail.com
9837,2021-10-23 14:57:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Viernes,CANDELARIA IIINO REPORTADOIV ENO REPORTADO19,URBANA,CASAS DE HABITACION,ARMA DE FUEGO,A PIE,A PIE,11.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,2004.0,35-279,houreg9939@unidatos.edu.co
9487,2021-10-11 02:30:00,CUNDINAMARCA,MADRID,Domingo,CENTRO,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,19.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,25430000.0,1996.0,21-547,kenpob8044@unidatos.edu.co
1234,2021-02-05 21:30:00,ANTIOQUIA,MEDELLÍN (CT),Jueves,NUEVOS CONQUISTADORES CNO REPORTADO13,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,16.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1999.0,97-208,lthgrk5089@gmail.com
9112,2021-09-29 00:47:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Martes,LOS MOLINOS DEL SUR ENO REPORTADO18,URBANA,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,40.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1975.0,32-730,gsnkrr5487@unidatos.edu.co


## 2. Fixing categorical column

In [62]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [63]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

<span style="color:red">TODO: Replace the values identified as error to a valid value from the dictionary.</span>

<span style="color:red">Hint: You can use the replace() pandas function.</span>

In [67]:
homicides_df = homicides_df.replace('SAN ANDRÉS', 'SAN ANDRÉS Y PROVIDENCIA')

In [68]:
homicides_df = homicides_df.replace('N. DE SANTANDER', 'NORTE DE SANTANDER')

In [69]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array([], dtype=object)

## 3. Analyzing potential duplicates

In [70]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep=False)]

In [71]:
duplicates_by_cedula.shape

(1627, 21)

In [72]:
duplicates_by_cedula.sort_values("CÉDULA", ascending=True).head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
4041,2021-05-03 23:30:00,CAUCA,EL TAMBO,Domingo,LA VICTORIA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,19256000.0,1970.0,10-048,aitufn1227@gmail.com
3810,2021-04-26 03:00:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co
693,2021-01-19 05:30:00,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co
10796,2021-11-20 03:00:00,VALLE,CALI (CT),Viernes,POTRERO GRANDE E21,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1994.0,10-079,iknnoj8430@unidatos.edu.co
2386,2021-03-14 02:00:00,META,FUENTE DE ORO,Sábado,VEREDA PUERTO NUEVO,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA BLANCA,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,50287000.0,1973.0,10-255,afompq7113@unidatos.edu.co
138,2021-01-02 06:20:00,CAUCA,SOTARA,Viernes,CENTRO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,22.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19760000.0,1993.0,10-255,cobgqs8819@unidatos.edu.co


<span style="color:red">TODO: Delete records with "CÉDULA" duplicated.</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

In [79]:
clear_df = homicides_df.drop_duplicates(subset=['CÉDULA'])
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula_2 = clear_df.loc[clear_df["CÉDULA"].duplicated(keep=False)]
duplicates_by_cedula_2.sort_values("CÉDULA", ascending=True).head(6)


,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO


## 4. Fixing formats

In [80]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [81]:
cedula_malformed.shape

(28, 21)

In [82]:
cedula_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
172,2021-01-03 11:00:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,680-21,oohghd8899@gmail.com
1114,2021-02-01 12:30:00,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,140-17,correo5853@colombia.gov.co
2119,2021-03-06 15:30:00,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,975-31,ohbqrk3631@unidatos.edu.co
3309,2021-04-11 10:30:00,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,348-66,pdkqur8407@unidatos.edu.co
3409,2021-04-13 23:20:00,CÓRDOBA,SAHAGÚN,Lunes,CORREGIMIENTO DE BAJO GRANDE,URBANA,BILLARES,CONTUNDENTES,A PIE,A PIE,49.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,23660000.0,1966.0,496-18,fdbbeo6751@unidatos.edu.co
3588,2021-04-19 20:00:00,BOLÍVAR,CARTAGENA (CT),Domingo,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,NO REPORTADO,NO REPORTADO,13001000.0,1970.0,188-03,diebuo5651@unidatos.edu.co


<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

In [94]:
cedula_malformed_2['CÉDULA'] = cedula_malformed['CÉDULA'].str.replace('-', '')
cedula_malformed_2['CÉDULA'] = cedula_malformed['CÉDULA'].str[:2] + '-' + cedula_malformed['CÉDULA'].str[2:]
cedula_malformed_2.head(6)

<ipython-input-94-87696ce6ef48>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cedula_malformed_2['CÉDULA'] = cedula_malformed['CÉDULA'].str.replace('-', '')
<ipython-input-94-87696ce6ef48>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cedula_malformed_2['CÉDULA'] = cedula_malformed['CÉDULA'].str[:2] + '-' + cedula_malformed['CÉDULA'].str[2:]


,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
172,2021-01-03 11:00:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,68-021,oohghd8899@gmail.com
1114,2021-02-01 12:30:00,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,14-017,correo5853@colombia.gov.co
2119,2021-03-06 15:30:00,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,97-531,ohbqrk3631@unidatos.edu.co
3309,2021-04-11 10:30:00,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,34-866,pdkqur8407@unidatos.edu.co
3409,2021-04-13 23:20:00,CÓRDOBA,SAHAGÚN,Lunes,CORREGIMIENTO DE BAJO GRANDE,URBANA,BILLARES,CONTUNDENTES,A PIE,A PIE,49.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,23660000.0,1966.0,49-618,fdbbeo6751@unidatos.edu.co
3588,2021-04-19 20:00:00,BOLÍVAR,CARTAGENA (CT),Domingo,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,NO REPORTADO,NO REPORTADO,13001000.0,1970.0,18-803,diebuo5651@unidatos.edu.co


<span style="color:red">TODO: Make something similar to check and fix the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

In [ ]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
mail_malformed = homicides_df.loc[homicides_df["CORREO"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

## 5. Automating imputation of categorical values

In [33]:
homicides_df["DIA"].unique()

array(['Jueves', 'Juees', 'Jueces', 'juves', 'Juevrs', 'Viernes',
       'Viermes', 'iernes', 'virnes', 'Vierens', 'Sábado', 'Sabadi',
       'Sabado', 'sábad', 'Sávado', 'Ssbado', 'Domingo', 'Domungo',
       'Doningo', 'domungo', 'Lunes', 'lune', 'Luns', 'Lumes', 'kunes',
       'Lnues', 'Martes', 'Mates', 'Marte', 'mates', 'Miércoles',
       'Miwrcoles', 'Mircoles', 'Voernes', 'domnigo', 'Maryes',
       'Miercoles', 'miércles', 'Dominog', 'Msrtes', 'Mirrcoles'],
      dtype=object)

In [34]:
# Calculating the distance between two words using the Levenshtein distance
pylev.levenshtein("sábado", "sabaod")

3

In [35]:
pylev.levenshtein("sábado", "viernes")

7

In [36]:
SequenceMatcher(None, "sábado", "sabaod").ratio()

0.6666666666666666

In [37]:
SequenceMatcher(None, "sábado", "viernes").ratio()

0.15384615384615385

<span style="color:red">How does SequenceMatcher works? How this differ from the Levenshtein distance?</span>

<span style="color:red">TODO: Create a function to fix the digitation errors for the "DIA" column.</span>

In [38]:
homicides_df["DIA"].unique()

array(['Jueves', 'Juees', 'Jueces', 'juves', 'Juevrs', 'Viernes',
       'Viermes', 'iernes', 'virnes', 'Vierens', 'Sábado', 'Sabadi',
       'Sabado', 'sábad', 'Sávado', 'Ssbado', 'Domingo', 'Domungo',
       'Doningo', 'domungo', 'Lunes', 'lune', 'Luns', 'Lumes', 'kunes',
       'Lnues', 'Martes', 'Mates', 'Marte', 'mates', 'Miércoles',
       'Miwrcoles', 'Mircoles', 'Voernes', 'domnigo', 'Maryes',
       'Miercoles', 'miércles', 'Dominog', 'Msrtes', 'Mirrcoles'],
      dtype=object)

## 6. Recalculation based on a different column

In [39]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].sample(10)

,AÑO DE NACIMIENTO,EDAD
1947,1980.0,35.0
2148,1998.0,17.0
9994,1993.0,22.0
7599,2001.0,14.0
3046,1976.0,39.0
2836,1984.0,31.0
6089,1970.0,45.0
4487,1958.0,57.0
5036,1998.0,17.0
10997,1962.0,53.0


<span style="color:red">TODO: Fix the "AÑO DE NACIMIENTO" column using the column "EDAD".</span>

## 7. Conclusion

<span style="color:red">Make a summary of the different data quality problems found on the dataset, the data quality dimension that is related to and the implemented strategy for solving or mitigating that specific problem.</span>